# Запитання: Як змінювались обсяги виробництва різних типів матеріалів впродовж даних років?



In [1]:
import altair as alt
import pandas as pd
import altair_viewer
import warnings
warnings.filterwarnings('ignore')
alt.renderers.enable('altair_viewer')

RendererRegistry.enable('altair_viewer')

In [2]:
data = pd.read_csv('UNdata_Export_20211219_151223778.csv')

data.head()

,Country or Area,Year,Commodity,Flow,Trade (USD),Weight (kg),Quantity Name,Quantity
0,Afghanistan,2018,"Cocoa powder, unsweetened",Import,7.247329e+04,8.070000e+04,Weight in kilograms,8.070000e+04
1,Afghanistan,2018,Chocolate/cocoa food preparations nes,Import,3.281740e+07,5.662823e+06,Weight in kilograms,5.662823e+06
2,Afghanistan,2018,Chocolate/cocoa food preparations nes,Export,4.327404e+06,0.000000e+00,Weight in kilograms,6.239175e+06
3,Afghanistan,2018,Chocolate/cocoa food preparations nes,Re-Export,7.452492e+04,0.000000e+00,Weight in kilograms,0.000000e+00
4,Afghanistan,2016,Chocolate/cocoa food preparations nes,Import,1.568728e+08,2.677301e+07,Weight in kilograms,2.677301e+07


In [3]:
data = data.groupby(['Commodity', 'Year' ]).mean().reset_index()
data.shape

(352, 5)

In [4]:
data.head()

,Commodity,Year,Trade (USD),Weight (kg),Quantity
0,Chocolate and other food preps containing coco...,1988,1.432337e+07,6.376257e+06,6.376257e+06
1,Chocolate and other food preps containing coco...,1989,9.745227e+06,5.253608e+06,5.253608e+06
2,Chocolate and other food preps containing coco...,1990,9.059725e+06,4.545472e+06,4.545472e+06
3,Chocolate and other food preps containing coco...,1991,1.029463e+07,5.459080e+06,5.459080e+06
4,Chocolate and other food preps containing coco...,1992,1.146687e+07,5.716594e+06,5.716594e+06


In [5]:
len(set(data['Commodity']))

11

In [6]:
data['year'] = data['Year'].astype(str)
options=[None]
options.extend(sorted(data.Commodity.unique().tolist()))
labels=['All']
labels.extend(sorted(data.Commodity.unique().tolist()))

In [76]:
input_dropdown = alt.binding_select(options = options, labels=labels)
select_sector = alt.selection_single(fields = ['Commodity'], bind = input_dropdown, name='Choose')
select_sector_click = alt.selection_single(on = 'click', fields = ['Commodity'], nearest = False, empty = 'all')

trend = alt.Chart(data, title='Trend of cocoa goods production').mark_line().encode(
    x = alt.X(field='year', 
              type='temporal', title='Year'
             ),
    y = alt.Y('Quantity:Q', title='Quantity of produced, kg'),
    
    tooltip = [
        alt.Tooltip('Commodity:O', title='Sector')
    ],
    detail = alt.Detail('Commodity:O'),
    size = alt.condition(
    (select_sector) | (select_sector_click),
    alt.value(3),
    alt.value(0.2)),
    color = alt.Color('Commodity:O'),
).add_selection(select_sector
).add_selection(select_sector_click
).transform_filter( (select_sector_click) | (select_sector)
).properties(width = 800, height = 400)

In [77]:
slider = alt.binding_range(name='Year:', min=min(data['Year']), max=max(data['Year']), step=1)
select_year =alt.selection_single(name="SelectorName", fields=['Year'],
                            bind=slider,init={"Year": 2010})


pie_chart = alt.Chart(data,  title='Cocoa goods production by sector for a given year').mark_arc().encode(
    theta=alt.Theta(field="Quantity", type="quantitative"),
    color=alt.Color(field="Commodity", type="nominal",scale=alt.Scale(scheme='set3')),
    tooltip = [
        alt.Tooltip('Commodity:O', title='Sector')
    ]
).add_selection(select_year
).add_selection(select_sector_click
).transform_filter( select_year
).properties(width = 400, height = 400)

In [78]:
alt.hconcat(pie_chart, 
            trend
           ).configure_view(
    strokeWidth=0
).configure_title(fontSize=23).configure_legend(
labelLimit = 0,
labelFontSize=12,
titleFontSize=17
) .configure_axis(
    labelFontSize=10,
    titleFontSize=17
)

Displaying chart at http://localhost:23288/

# Запитання: Як змінюється ціна за вироблення товару в межах певної категорії?

In [10]:
data['$/kg'] = data['Trade (USD)']/data['Quantity']
data.head()

,Commodity,Year,Trade (USD),Weight (kg),Quantity,year,$/kg
0,Chocolate and other food preps containing coco...,1988,1.432337e+07,6.376257e+06,6.376257e+06,1988,2.246360
1,Chocolate and other food preps containing coco...,1989,9.745227e+06,5.253608e+06,5.253608e+06,1989,1.854959
2,Chocolate and other food preps containing coco...,1990,9.059725e+06,4.545472e+06,4.545472e+06,1990,1.993132
3,Chocolate and other food preps containing coco...,1991,1.029463e+07,5.459080e+06,5.459080e+06,1991,1.885781
4,Chocolate and other food preps containing coco...,1992,1.146687e+07,5.716594e+06,5.716594e+06,1992,2.005892


In [72]:

input_dropdown = alt.binding_select(options = options, labels=labels)
select_sector = alt.selection_single(fields = ['Commodity'], bind = input_dropdown, name='Choose')

histogram_context  = alt.Chart(data).mark_bar(color = 'lightgray').encode(
    x = alt.X("$/kg:Q", bin=alt.Bin(extent=[0, 7], step=0.25), title='Price per kg of goods, $'),
    y = alt.Y('count()')
)

hist = alt.Chart(data, title='Distribution of price for kilo of cocoa goods over sectors').mark_bar().encode(
    x = alt.X("$/kg:Q",bin=alt.Bin(extent=[0, 7], step=0.25), title='Price per kg of goods, $'),
    y = alt.Y('count()'),
    color = alt.value('FC9C3C')
).add_selection(select_sector
).transform_filter( select_sector
)

histogram = (histogram_context+hist).properties(width = 800, height = 400
).configure_title(fontSize=23).configure_axis(
    labelFontSize=10,
    titleFontSize=17
)
histogram

Displaying chart at http://localhost:23288/

In [42]:
len(set(data['Commodity']))

11